# Guns N' Google

## More In-Depth Analysis of the Top 10 Results

In [1]:
SIMPLE_QUERIES = ("gun", "firearm", "gun control", "gun rights", "mass shooting")

### Manual Analysis of the Search Results

In [2]:
STOP = 10
from googlesearch import search

search_results = {}
# for q in SIMPLE_QUERIES:
#     search_results[q] = list(search(q, tld='com', lang='en', stop=STOP, only_standard=True))

Store

In [3]:
from os.path import join
import pandas as pd
# for q in SIMPLE_QUERIES:
#     df = pd.DataFrame({
#         'url': pd.Series(search_results[q]),
#         'program': pd.Categorical(['todo']*len(search_results[q])),
#         'type': pd.Categorical(['todo']*len(search_results[q])),
#         'source title': pd.Series(['todo']*len(search_results[q])),
#     })
#     df.to_csv(join("data", "{name}.csv".format(name=q)))

TODO: Analysed the 10 first results manually, by reading the article and not the source/domain. labeled Pro-Control if the article was leaning toward more gun control, Anti-Control if it was supporting gun rights, Balanced if comments from both views were reflected in a balanced manner, and N/A if the article hadn't mentioned the debate. We also categorized the articles into these categories: News Article, Blog Post, Research Report, Encyclopedia Entry, Database, Shop, Advocacy Group, and List of Articles. We ignored the cards in the search results page.

In [17]:
from os.path import join
import pandas as pd

reviewed_search_results = {}
for q in SIMPLE_QUERIES:
    reviewed_search_results[q] = pd.read_csv(join("data", "{name}.reviewed.csv".format(name=q)), usecols=['url', 'program', 'type', 'source title'])

In [18]:
import helper
helper.visualize_reviewed_search_results(reviewed_search_results)

,gun,firearm,gun control,gun rights,mass shooting
0,Wikipedia(Encyclopedia Entry),Wikipedia(Encyclopedia Entry),Wikipedia(Encyclopedia Entry),OpenSecrets.org(Advocacy Group),CNN(News Article)
1,Sportsman’s Guide(Shop),Cabela's(Shop),ProCon.org(Forum),NPR(List of Articles),CNN(News Article)
2,Academy(Shop),Academy(Shop),Axios(News Article),U.S. News(News Article),Independent(News Article)
3,NBC News(News Article),Cheaper Than Dirt(Shop),Vox(News Article),Wikipedia(Encyclopedia Entry),ABC News(News Article)
4,The New York Times(News Article),Merriam-Webster(Encyclopedia Entry),The New York Times(News Article),The Trace(List of Articles),CBS News(News Article)
5,The Washington Post(News Article),ATF.gov(Government Guidelines),Twitter(Forum),The Oregonian(News Article),Wikipedia(Encyclopedia Entry)
6,The New York Times(News Article),ATF.gov(Government Guidelines),Just Facts(Encyclopedia Entry),Congressman Robert B. Aderholt(Blog Post),The Washington Post(News Article)
7,Pew Research Center(Research Report),ATF.gov(Government Guidelines),The New York Times(List of Articles),The Atlantic(News Article),Mass Shootings(Database)
8,Wired(News Article),Scientific American(Research Report),The Council on Foreign Relations(Blog Post),The New York Times(News Article),The Washington Post(Blog Post)


TODO: post description

### Analysing the Sources
TODO: Lippmanian device: lists of pro word and antiword per issues (ex: gun control, gun right)

In [6]:
PROCONTROL_KEYWORD = 'mass-shooting  "background check"'
ANTICONTROL_KEYWORD = 'mass-shooting  "gun rights"'

from googlesearch import hits, get_random_user_agent
import time
ua = get_random_user_agent()
PROCONTROL_KEYWORD_HITS = hits(PROCONTROL_KEYWORD, tld='com', lang='en', user_agent=ua)
time.sleep(5)
ANTICONTROL_KEYWORD_HITS = hits(ANTICONTROL_KEYWORD, tld='com', lang='en', user_agent=ua)

In [7]:
from urllib.parse import urlparse

source_domains = set([])
for q in SIMPLE_QUERIES:
    source_domains |= set([urlparse(url).netloc for url in reviewed_search_results[q].url])

In [8]:
source_normalized_usage_procontrol = {}
source_normalized_usage_anticontrol = {}

import json
from os.path import join

USE_STORED_VALUES = True
if USE_STORED_VALUES:
    with open(join('data', 'source_normalized_usage_procontrol.json')) as infile:
        source_normalized_usage_procontrol = json.load(infile)
    with open(join('data', 'source_normalized_usage_anticontrol.json')) as infile:
        source_normalized_usage_anticontrol = json.load(infile)

import time, random
from urllib.error import HTTPError
from googlesearch import hits, get_random_user_agent
ua = get_random_user_agent()
for domain in source_domains:
    if domain in source_normalized_usage_procontrol:
        continue
    while True:
        try:
            time.sleep(10 + random.random() * 100) # To avoid Google captcha
            procontrol_q = 'site:{domain} {keyword}'.format(domain=domain, keyword=PROCONTROL_KEYWORD)
            source_normalized_usage_procontrol[domain] = hits(
                procontrol_q, tld='com', lang='en', user_agent=ua) / PROCONTROL_KEYWORD_HITS
            time.sleep(10 + random.random() * 100) # To avoid Google captcha
            anticontrol_q = 'site:{domain} {keyword}'.format(domain=domain, keyword=ANTICONTROL_KEYWORD)
            source_normalized_usage_anticontrol[domain] = hits(
                anticontrol_q, tld='com', lang='en', user_agent=ua) / ANTICONTROL_KEYWORD_HITS
            break # The infinite retry loop
        except HTTPError as e:
            if e.code != 503:
                raise e
            ua = get_random_user_agent()
            time.sleep(600)

# Store
with open(join('data', 'source_normalized_usage_procontrol.json'), 'w') as outfile:
    json.dump(source_normalized_usage_procontrol, outfile, sort_keys=True,
              indent=4, separators=(',', ': '))
with open(join('data', 'source_normalized_usage_anticontrol.json'), 'w') as outfile:
    json.dump(source_normalized_usage_anticontrol, outfile, sort_keys=True,
              indent=4, separators=(',', ': '))

In [9]:
max_procontrol_value = max(source_normalized_usage_procontrol.values())
max_anticontrol_value = max(source_normalized_usage_anticontrol.values())
source_score_procontrol = {d: v / max_procontrol_value for (d, v) in source_normalized_usage_procontrol.items()}
source_score_anticontrol = {d: v / max_anticontrol_value for (d, v) in source_normalized_usage_anticontrol.items()}

In [19]:
import helper
helper.visualize_source_analysis_results(reviewed_search_results, source_score_procontrol, source_score_anticontrol)

,gun,firearm,gun control,gun rights,mass shooting
0,Wikipedia(Encyclopedia Entry),Wikipedia(Encyclopedia Entry),Wikipedia(Encyclopedia Entry),OpenSecrets.org(Advocacy Group),CNN(News Article)
1,Sportsman’s Guide(Shop),Cabela's(Shop),ProCon.org(Forum),NPR(List of Articles),CNN(News Article)
2,Academy(Shop),Academy(Shop),Axios(News Article),U.S. News(News Article),Independent(News Article)
3,NBC News(News Article),Cheaper Than Dirt(Shop),Vox(News Article),Wikipedia(Encyclopedia Entry),ABC News(News Article)
4,The New York Times(News Article),Merriam-Webster(Encyclopedia Entry),The New York Times(News Article),The Trace(List of Articles),CBS News(News Article)
5,The Washington Post(News Article),ATF.gov(Government Guidelines),Twitter(Forum),The Oregonian(News Article),Wikipedia(Encyclopedia Entry)
6,The New York Times(News Article),ATF.gov(Government Guidelines),Just Facts(Encyclopedia Entry),Congressman Robert B. Aderholt(Blog Post),The Washington Post(News Article)
7,Pew Research Center(Research Report),ATF.gov(Government Guidelines),The New York Times(List of Articles),The Atlantic(News Article),Mass Shootings(Database)
8,Wired(News Article),Scientific American(Research Report),The Council on Foreign Relations(Blog Post),The New York Times(News Article),The Washington Post(Blog Post)
